## CESM2 - LARGE ENSEMBLE (LENS2)

- This Notebook aims to visualize the AMOC in different sectors of the Atlantic Ocean 

### Imports

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np  
import dask
import cf_xarray
import intake
import cftime
import nc_time_axis
import intake_esm
import matplotlib.pyplot as plt
import pop_tools
from dask.distributed import Client
from ncar_jobqueue import NCARCluster
import warnings, getpass, os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.integrate import cumtrapz
from scipy import integrate
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

### Dask

In [ ]:
mem_per_worker = 20 # memory per worker in GB 
num_workers = 40 # number of workers
cluster = NCARCluster(cores=1, processes=3, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=6:mem={mem_per_worker}GB', walltime='8:00:00')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

### Read in OGCM history file & MOC template file

In [ ]:
catalog = intake.open_esm_datastore(
    '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
)

In [ ]:
cat_subset = catalog.search(component='ocn',variable=['MOC'],frequency='month_1')

In [ ]:
# Load catalog entries for subset into a dictionary of xarray datasets
dset_dict_raw  = cat_subset.to_dataset_dict(zarr_kwargs={'consolidated': True}, storage_options={'anon': True})
print(f'\nDataset dictionary keys:\n {dset_dict_raw.keys()}')

In [ ]:
pd=('historical','ssp370')                 # Periods (historical and projection)
ff=('cmip6','smbb')                        # Forcings
fb=(['MOC']) # Variable.

ds_dict = dict()
for var in fb: # We used loops, because we keep the same structure even if other variables are added
    # 1- combine historical and ssp370 (concatenate in time)
    ds_dict_tmp = dict()
    for scenario in ff:
        ds_dict_tmp[scenario] = xr.combine_nested([dset_dict_raw[f'ocn.historical.pop.h.{scenario}.{var}'], dset_dict_raw[f'ocn.ssp370.pop.h.{scenario}.{var}']],concat_dim=['time'])
        
        # 2- combine cmip6 and smbb (concatenate in member_id)
    ds_dict[var] = xr.combine_nested([ds_dict_tmp['cmip6'], ds_dict_tmp['smbb']], concat_dim=['member_id'])
    #del ds_dict_tmp

In [ ]:
%%time
Z1 = ds_dict[var][var].isel(
    transport_reg=1).sel(
    lat_aux_grid='-34.5',method='nearest').sum(dim='moc_comp').resample(
    time='1Y', closed='left').mean('time').load()

Z1=(Z1-Z1.isel(time=0)).mean('member_id')

Z2 = ds_dict[var][var].isel(
    transport_reg=1).sel(
    lat_aux_grid='-17.25',method='nearest').sum(dim='moc_comp').resample(
    time='1Y', closed='left').mean('time').load()

Z2=(Z2-Z2.isel(time=0)).mean('member_id')

Z3 = ds_dict[var][var].isel(
    transport_reg=1).sel(
    lat_aux_grid='0',method='nearest').sum(dim='moc_comp').resample(
    time='1Y', closed='left').mean('time').load()

Z3=(Z3-Z3.isel(time=0)).mean('member_id')

In [ ]:
Z1.coords['moc_z']=Z1.coords['moc_z']*-0.01
Z2.coords['moc_z']=Z2.coords['moc_z']*-0.01
Z3.coords['moc_z']=Z3.coords['moc_z']*-0.01
label=('$\Delta$(Sv)')

In [ ]:
def split_depth_plot(da, 
                     ax_top, 
                     ax_bottom, 
                     depth):
    # Make sure the colour range is the same for the top and bottom plots
    mn = Z1.sel(time=slice('1850-01-01','2100-12-31')).min()
    mx = Z1.sel(time=slice('1850-01-01','2100-12-31')).max()
    if np.abs(mn)>np.abs(mx):
        mx=-mn
    else: 
        mn=-mx
    
    orig_map=plt.cm.get_cmap('RdBu')
    scale_cor=orig_map.reversed()
    N = 50
    
    # Plot the top half
    da.sel(moc_z=slice(0,depth)).plot.contourf('time','moc_z', vmin=mn, vmax=mx,
                                                add_colorbar=False, 
                                                levels=np.linspace(mn, mx, N),
                                                ax=ax_top, cmap=scale_cor, add_labels=False) 
    CS=da.sel(moc_z=slice(0,depth)).plot.contour('time','moc_z', vmin=mn, vmax=mx,
                                            add_colorbar=False, 
                                            levels = [mn*0.05,0,mx*0.05],
                                            ax=ax_top, #colors=('k',),
                                            kwargs=dict(inline=True),
                                            colors=['blue','black','red'],
                                            add_labels=True)
    ax_bottom.clabel(CS)
    
    # Remove x axis labels from the top
    #ax_top.tick_params(axis='x', bottom=False, labelbottom=False)
    
    
    # Plot the bottom half
    im=da.sel(moc_z=slice(depth,None)).plot.contourf('time','moc_z', vmin=mn, vmax=mx,
                                                add_colorbar=False, 
                                                levels=np.linspace(mn, mx, N),
                                                ax=ax_bottom, cmap=scale_cor, add_labels=False)
    
    CS=da.sel(moc_z=slice(depth,None)).plot.contour('time','moc_z', vmin=mn, vmax=mx,
                                               add_colorbar=False, 
                                               levels = [-2.6,-1.8,0,1.8,2.6],
                                            colors=['darkblue','darkcyan','black','darkorange','darkred'],
                                               kwargs=dict(inline=True),
                                               ax=ax_bottom, add_labels=True)
    ax_bottom.clabel(CS)
    plt.grid(color='gray', linestyle='-', linewidth=0.7)

    return im, mx, mn

In [ ]:
# Set the display

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[22, 10])

gs = matplotlib.gridspec.GridSpec(2, 3, hspace=0.03, wspace=0.01, height_ratios=(2, 4))

# Display the plots

### South Atlantic 
st=f'split_depth_plot(Z1, plt.subplot(gs[0,0]), plt.subplot(gs[1,0]), depth=-150)'
exec(st)

### Western Side
st=f'split_depth_plot(Z2, plt.subplot(gs[0,1]), plt.subplot(gs[1,1]), depth=-150)'
exec(st)

### Eastern Side
st=f'im, mx, mn=split_depth_plot(Z3, plt.subplot(gs[0,2]), plt.subplot(gs[1,2]), depth=-150)'
exec(st)

# Set plots

ax1=plt.subplot(gs[0,0]); ax4=plt.subplot(gs[1,0]);
ax2=plt.subplot(gs[0,1]); ax5=plt.subplot(gs[1,1]);
ax3=plt.subplot(gs[0,2]); ax6=plt.subplot(gs[1,2]);

ax1.grid(color='gray', linestyle='-', linewidth=0.7)
ax2.grid(color='gray', linestyle='-', linewidth=0.7)
ax3.grid(color='gray', linestyle='-', linewidth=0.7)

# Set Colorbar

axins = inset_axes(ax6,
                   width="5%",  # width = 5% of parent_bbox width
                   height="100%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(1.05, 0, 1, 1.52),
                   bbox_transform=ax6.transAxes,
                   borderpad=0,
                   )

orig_map=plt.cm.get_cmap('RdBu')
scale_cor=orig_map.reversed()

ticks = np.linspace(mn, mx, 10, endpoint=True)

cbar=fig.colorbar(im,cax=axins, ticks=ticks, cmap=scale_cor, label=label, extend='neither')

# Set axis labels

ax4.set_xlabel('Time [Years]'), ax5.set_xlabel('Time [Years]'); ax6.set_xlabel('Time [Years]')

ax1.set_ylabel('Depth [m]'), ax4.set_ylabel('Depth [m]')

# set labels

ax4.set_title(None); ax5.set_title(None); ax6.set_title(None) 

ax1.xaxis.set_ticklabels([]); ax2.xaxis.set_ticklabels([]); ax3.xaxis.set_ticklabels([])

ax3.set_yticklabels([]); ax2.set_yticklabels([]); ax5.set_yticklabels([]); ax6.set_yticklabels([])

ax5.set(ylabel=None); ax6.set(ylabel=None)

ax2.set(ylabel=None); ax3.set(ylabel=None)

# set titles

#ax1.set_title('South Atlantic'); ax2.set_title('Western side'); ax3.set_title('Eastern side') 

# save

st=f'plt.savefig(\'Hovmoller_Diagram_MOC.png\',dpi=300,bbox_inches=\'tight\')'
exec(st)
plt.show()

### Vertical profile

In [ ]:
dz=ds_MOC.dz